In [24]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import pickle as p
import itertools

##### LOAD FEATURES

Carrega o train set com a features e deleta possíveis nulos

In [3]:
train_features = pd.read_csv('quora_train_engineered.csv')

In [4]:
print(len(train_features))
train_features.head(1)

404290


,question1,question2,is_duplicate,len_q1,len_q2,diff_len,len_char_q1,len_char_q2,len_word_q1,len_word_q2,...,cityblock_distance,jaccard_distance,canberra_distance,euclidean_distance,minkowski_distance,braycurtis_distance,skew_q1vec,skew_q2vec,kur_q1vec,kur_q2vec
0,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0,66,57,9,20,20,14,12,...,5.081614,1.0,94.023324,0.371408,0.168999,0.186557,0.031817,-0.091902,0.050416,0.337301


In [5]:
train_features = train_features.dropna()
print(len(train_features))

402489


In [6]:
train_features = train_features.replace([np.inf, -np.inf], np.nan).dropna()
print(len(train_features))

393325


In [7]:
train_features.describe()

,is_duplicate,len_q1,len_q2,diff_len,len_char_q1,len_char_q2,len_word_q1,len_word_q2,common_words,fuzz_qratio,...,cityblock_distance,jaccard_distance,canberra_distance,euclidean_distance,minkowski_distance,braycurtis_distance,skew_q1vec,skew_q2vec,kur_q1vec,kur_q2vec
count,393325.000000,393325.000000,393325.000000,393325.000000,393325.000000,393325.000000,393325.000000,393325.000000,393325.000000,393325.000000,...,393325.000000,393325.000000,393325.000000,393325.000000,393325.000000,393325.000000,393325.000000,393325.000000,393325.000000,393325.000000
mean,0.373147,60.311986,60.915289,-0.603303,20.358383,20.346022,11.087015,11.333958,4.577741,58.347913,...,9.168629,0.931813,131.565848,0.662708,0.298953,0.367942,0.010352,0.010401,-0.052685,-0.054127
std,0.483641,29.816552,33.826257,32.729612,4.103479,4.232956,5.403554,6.298609,3.106880,21.973585,...,4.264971,0.252065,47.306981,0.308122,0.139067,0.195305,0.135298,0.134904,0.265730,0.264215
min,0.000000,1.000000,2.000000,-1080.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.669451,-0.669451,-0.857844,-0.829341
25%,0.000000,40.000000,40.000000,-12.000000,18.000000,17.000000,8.000000,7.000000,2.000000,41.000000,...,6.601517,1.000000,112.802617,0.477007,0.215150,0.245304,-0.080458,-0.080101,-0.238232,-0.239380
50%,0.000000,52.000000,51.000000,0.000000,20.000000,20.000000,10.000000,10.000000,4.000000,59.000000,...,9.194268,1.000000,138.476616,0.664615,0.299724,0.351586,0.011966,0.011953,-0.081463,-0.082807
75%,1.000000,72.000000,72.000000,12.000000,23.000000,23.000000,13.000000,13.000000,6.000000,76.000000,...,11.827348,1.000000,160.990508,0.854588,0.385468,0.471933,0.101535,0.100640,0.101924,0.100228
max,1.000000,623.000000,1169.000000,487.000000,52.000000,55.000000,125.000000,237.000000,41.000000,100.000000,...,20.511668,1.000000,230.190867,1.496936,0.682163,1.101037,0.675833,0.781670,2.288984,2.227666


In [8]:
train_data = train_features.values

##### Normalize

In [9]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train = scaler.fit_transform(train_data[0::,3::])

y_train = train_data[0::, 2]

/usr/local/lib/python3.5/dist-packages/sklearn/utils/validation.py:429: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, _DataConversionWarning)


In [10]:
print(X_train)
print(set(y_train))
print(np.shape(X_train))
print(np.shape(y_train))

[[ 0.19076723 -0.11574718  0.29341364 ..., -0.75833929  0.38799228
   1.48147671]
 [-0.31230969  0.80070182 -1.11204323 ...,  0.62491078  1.26705555
   0.07449748]
 [ 0.42553645 -0.05662144  0.4461807  ...,  0.99443795  0.29896417
  -1.58963206]
 ..., 
 [-1.45261735 -1.29826201  0.01843295 ..., -1.16928459 -0.85365441
  -1.30004414]
 [ 1.12984413  1.95365378 -0.98982959 ..., -1.22979346 -1.24111792
  -0.41197355]
 [-0.78184814 -0.47050163 -0.22599433 ...,  1.31754859  2.03282159
   2.04993959]]
{0, 1}
(393325, 28)
(393325,)


##### Import Metrics

In [11]:
from sklearn import model_selection

from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.metrics import accuracy_score
from sklearn.metrics import average_precision_score
from sklearn.metrics import f1_score

In [22]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    
def clf_eval(clf, X, y_true, classes=['Não Repetidas', 'Repetidas']):
    y_pred = clf.predict(X)
    y_pred = np.asarray(y_pred, dtype=np.int)
    y_true = np.asarray(y_true, dtype=np.int)
    for i in range(len(y_pred)):
        if type(y_pred[i]) != type(y_true[i]):
            print(type(y_pred[i]), type(y_true[i]))
            break
    clf_matrix = confusion_matrix(y_true, y_pred)
    print('Classification Report')
    print(classification_report(y_true, y_pred, target_names=classes))
    print('ROC Score: {}'.format(roc_auc_score(y_true, y_pred)))
    print('Accuracy Score: {}'.format(accuracy_score(y_true, y_pred)))
    print('Average Precision Score: {}'.format(average_precision_score(y_true, y_pred)))
    print('f1 Score: {}'.format(f1_score(y_true, y_pred)))
    plot_confusion_matrix(clf_matrix, classes=classes)
    return roc_auc_score(y_true, y_pred)

##### Primeira tentativa

Cross Validation e XGBoost 

Sem tuning dos parametros

In [14]:
X_traincv, X_testcv, y_traincv, y_testcv = model_selection.train_test_split(X_train,
                                                                            y_train,
                                                                            test_size=0.3,
                                                                            random_state=0)
print(len(X_traincv), len(X_testcv), len(y_traincv), len(y_testcv))

275327 117998 275327 117998


In [15]:
y_traincv = np.asarray(y_traincv, dtype="|S6")

In [18]:
import xgboost

clf_xgb1 = xgboost.sklearn.XGBClassifier(base_score=0.5,
                                        learning_rate=0.1,
                                        n_estimators=250,
                                        max_delta_step=0,
                                        max_depth=3,
                                        min_child_weight=1,
                                        missing=None,
                                        gamma=0,
                                        subsample=1,
                                        colsample_bylevel=1,
                                        colsample_bytree=1,
                                        objective= 'binary:logitraw',
                                        #objective='multi:softprob',
                                        reg_alpha=0, 
                                        reg_lambda=1,
                                        nthread=-1,
                                        scale_pos_weight=1,
                                        seed=0,
                                        silent=False,).fit(X_traincv, y_traincv)
print(len(clf_xgb.predict(X_testcv)))
print(len(y_testcv))

117998
117998


In [25]:
roc_xgb = clf_eval(clf_xgb1, X_testcv, y_testcv)

Classification Report
               precision    recall  f1-score   support

Não Repetidas       0.70      0.93      0.80     74095
    Repetidas       0.72      0.32      0.44     43903

  avg / total       0.71      0.70      0.66    117998

ROC Score: 0.6224522738989107
Accuracy Score: 0.7002830556450109
Average Precision Score: 0.6459209678723716
f1 Score: 0.44140131412686373
Confusion matrix, without normalization
[[68659  5436]
 [29930 13973]]
